主要介绍Numpy的genfromtxt函数

In [54]:
import numpy as np
from io import StringIO, BytesIO

In [2]:
"""
genfromtxt(fname, dtype=float, comments='#', delimiter=None,
               skip_header=0, skip_footer=0, converters=None,
               missing_values=None, filling_values=None, usecols=None,
               names=None, excludelist=None, deletechars=None,
               replace_space='_', autostrip=False, case_sensitive=True,
               defaultfmt="f%i", unpack=None, usemask=False, loose=True,
               invalid_raise=True, max_rows=None, encoding='bytes')
               
dtype: 数据类型，可逐列指定，默认为float
comments: 指定comments符号，从而不导入。默认为#
delimiter: 分隔符， 如csv中的, 或;. 默认为空白区域（包括制表符）
skip_header: 跳过前序行
skip_footer: 跳过后续行
converters: 指定列的转换器
missing_values: 指定列的缺失值
filling_values: 指定列的缺失填充值
usecols: 指定载入列
names: 指定列名，从而形成异构的ndarrary,  若为True，选择skip_header后的第一行作为names。该参数优先级高于comments参数
autostrip: 自动清楚数值间的空格
unpack：拆解
usemask： 是否返回掩膜数组
max_rows：最大行数，注意不要与skip_footer重复
"""
print(" ")

### delimiter参数

In [15]:
data = "1, 2, 3\n4, 5, 6"
np.genfromtxt(StringIO(data), delimiter=",")

array([[1., 2., 3.],
       [4., 5., 6.]])

In [17]:
# delimiter还可以指定宽度，用于解决固定宽度文件的读入问题
data = "  1  2  3\n  4  5 67\n890123  4"
np.genfromtxt(StringIO(data), delimiter=3)

array([[  1.,   2.,   3.],
       [  4.,   5.,  67.],
       [890., 123.,   4.]])

In [20]:
# delimiter还可以指定各列大小不同的问题
data = "123456789\n   4  7 9\n   4567 9"
np.genfromtxt(StringIO(data), delimiter=(4, 3,2), autostrip=True)

array([[1234.,  567.,   89.],
       [   4.,    7.,    9.],
       [   4.,  567.,    9.]])

### autostrip参数

默认情况下，当一行被分解为一系列字符串时，单个条目不会被剥离前导空白或尾随空白。

In [28]:
# 可见字符串前后的空白被保留
data = "1, abc , 2\n 3, xxx, 4"
np.genfromtxt(StringIO(data), delimiter=",", dtype='1S5')

array([[b'1', b' abc ', b' 2'],
       [b'3', b' xxx', b' 4']], dtype='|S5')

In [29]:
np.genfromtxt(StringIO(data), delimiter=",", autostrip=True, dtype='1S5')

array([[b'1', b'abc', b'2'],
       [b'3', b'xxx', b'4']], dtype='|S5')

### comment参数

用于定义标记注释开始的字符串

In [35]:
data = """#
# Skip me !
# Skip me too !
1, 2
3, 4
5, 6 #This is the third line of the data
7, 8
# And here comes the last line
 9, 0
"""

np.genfromtxt(StringIO(data), delimiter=',', comments='#')

array([[1., 2.],
       [3., 4.],
       [5., 6.],
       [7., 8.],
       [9., 0.]])

### skip_header和skip_footer参数

默认均为0，即不跳过

In [39]:
data = '\n'.join(str(i) for i in range(10))
data

'0\n1\n2\n3\n4\n5\n6\n7\n8\n9'

In [40]:
np.genfromtxt(StringIO(data))

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [41]:
np.genfromtxt(StringIO(data),skip_header=3, skip_footer=2)

array([3., 4., 5., 6., 7.])

### usecols参数和unpack参数

选择敢兴趣的列，并选择是否拆分

In [42]:
data = "1 2 3\n4 5 6"
np.genfromtxt(StringIO(data), usecols=(0, -1))

array([[1., 3.],
       [4., 6.]])

In [43]:
a, b = np.genfromtxt(StringIO(data), usecols=(0, -1), unpack=True)
a 

array([1., 4.])

In [45]:
# 如果列有名字，也可以在usecols中指定名字
data = "1 2 3\n4 5 6"
np.genfromtxt(StringIO(data), names="a, b, c", usecols=("a", "c"))

array([(1., 3.), (4., 6.)], dtype=[('a', '<f8'), ('c', '<f8')])

### names参数

In [46]:
# 可以通过直接生命结构化的dtype来指定name
data = "1 2 3\n 4 5 6"
np.genfromtxt(StringIO(data), dtype=[(_, int) for _ in 'abc'])

array([(1, 2, 3), (4, 5, 6)],
      dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])

In [47]:
# 可以通过显示的指定names
data = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(data, names="A, B, C")   # 也可以传递列表

array([(1., 2., 3.), (4., 5., 6.)],
      dtype=[('A', '<f8'), ('B', '<f8'), ('C', '<f8')])

In [51]:
# 或者通过names=True直接从原文件中获取列名
data = StringIO("So it goes\na b c\n1 2 3\n 4 5 6")
np.genfromtxt(data,skip_header=1, names=True)   

array([(1., 2., 3.), (4., 5., 6.)],
      dtype=[('a', '<f8'), ('b', '<f8'), ('c', '<f8')])

### converters参数

加载数据时直接对指定列进行转化，该参数值通常为以列索引或列名称为key的字典

In [52]:
data = "1, 2.3%, 45.\n6, 78.9%, 0"
names = ("i", "p", "n")
np.genfromtxt(StringIO(data), delimiter=",", names=names)

array([(1., nan, 45.), (6., nan,  0.)],
      dtype=[('i', '<f8'), ('p', '<f8'), ('n', '<f8')])

In [76]:
data = "1, 2.3%, 45.\n6, 78.9%, 0"
convertfunc = lambda x: float(x.decode('utf-8').strip("%"))/100    # 注意要decode
np.genfromtxt(StringIO(data), dtype=(np.int, np.string_, np.int), delimiter=",", names=names, converters={"p":convertfunc})

array([(1, b'', -1), (6, b'',  0)],
      dtype=[('i', '<i8'), ('p', 'S'), ('n', '<i8')])

### missing_values和 filling_values=None

可以传递 有,的字符串， 数组或字典（字典为列索引或列名）

In [77]:
data = "N/A, 2, 3\n4, ,???"
kwargs = dict(delimiter=",",
              dtype=int,
              names="a,b,c",
              missing_values={0:"N/A", 'b':" ", 2:"???"},
              filling_values={0:0, 'b':0, 2:-999})
np.genfromtxt(StringIO(data), **kwargs)

array([(0, 2,    3), (4, 0, -999)],
      dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])

### usemask参数

通过构造一个布尔掩码来跟踪丢失数据的发生

In [79]:
data = "N/A, 2, 3\n4, ,???"
kwargs = dict(delimiter=",",
              dtype=int,
              names="a,b,c",
              missing_values={0:"N/A", 'b':" ", 2:"???"},
              usemask=True)
np.genfromtxt(StringIO(data), **kwargs)

masked_array(data=[(--, 2, 3), (4, --, --)],
             mask=[( True, False, False), (False,  True,  True)],
       fill_value=(999999, 999999, 999999),
            dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])